In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 632.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Tool

In [ ]:
from langchain_core.tools import tool
import requests
import os

@tool
def search_news_by_keyword(keyword: str, region: str = "en-US") -> str:
    """
    Search Google News for articles matching a keyword using the google-news13 API.

    Args:
        keyword (str): The keyword to search for.
        region (str): Language-region code (default: 'en-US').

    Returns:
        str: A summary of the top 3 news articles with title, summary, and link.
    """
    url = "https://google-news13.p.rapidapi.com/search"
    querystring = {"keyword": keyword, "lr": region}
    headers = {
        "x-rapidapi-key": "api key",
        "x-rapidapi-host": "google-news13.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        items = data.get("items", [])

        if not items:
            return f"No news found for keyword: {keyword}"

        result = f"📰 Top News for '{keyword}':\n"
        for idx, item in enumerate(items[:3], start=1):  # Top 3 items
            title = item.get("title", "No title")
            snippet = item.get("snippet", "No summary")
            link = item.get("newsUrl", "No link")
            result += f"\n{idx}. {title}\n📄 {snippet}\n🔗 {link}\n"
        return result.strip()

    except requests.exceptions.RequestException as e:
        return f"❌ Error fetching news: {str(e)}"
    except (KeyError, TypeError) as e:
        return f"❌ Unexpected response format: {str(e)}"


# Agent

In [ ]:
google_assistant = create_react_agent(
    model=model,
    tools=[search_news_by_keyword],
    prompt="You are a helpful Google search assistant. Your job is to help users find Google articles based on their search queries.",
    name="google_assistant"
)

# Agent Invoke

In [ ]:
response = google_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="mental health")
            ]
    }
    )
response["messages"]

[HumanMessage(content='mental health', additional_kwargs={}, response_metadata={}, id='f8ca895c-73fc-44a7-8caa-7ef221ecdf6f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xE835aT8INIGtmAUvc7cvatU', 'function': {'arguments': '{"keyword":"mental health"}', 'name': 'search_news_by_keyword'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 139, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bgw9c5SCnNJI080pkxBaOni9Wl59x', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='google_assistant', id='run--d7760d33-0f5f-4e73-a019-fb3a2f799fee-0', tool_calls=[{'name': 'search_news_by_keyword', 'args': {

In [ ]:
response["messages"][-1].content

"Here are the top articles related to mental health:\n\n1. **NHS trust cleared of manslaughter after woman takes life in mental health unit**\n   - Alice Figueiredo died at Goodmayes Hospital in Redbridge in 2015.\n   - [Read more](https://www.independent.co.uk/news/uk/home-news/alice-figueiredo-nhs-death-suicide-manslaughter-trial-b2766617.html)\n\n2. **Eliza Dushku Graduates with Master’s Degree in Clinical Mental Health Counseling**\n   - Eliza Dushku graduated with her master's degree in clinical mental health counseling from Lesley University in Cambridge, Mass., on June 6.\n   - [Read more](https://people.com/eliza-dushku-graduates-masters-degree-clinical-mental-health-counseling-lesley-university-11750164)\n\n3. **Mothers report worse mental health in new report**\n   - The decline in maternal mental health occurred across all sociodemographic groups in a recent study.\n   - [Read more](https://www.washingtonpost.com/health/2025/06/09/mothers-mental-health-decline-parenting/)\n\

# Class Local

# Google News Tool. PY

In [ ]:
from langchain_core.tools import tool
import requests
import os
from config.setting import RAPIDAPI_KEY
@tool
def search_news_by_keyword(keyword: str, region: str = "en-US") -> str:
    """
    Search Google News for articles matching a keyword using the google-news13 API.

    Args:
        keyword (str): The keyword to search for.
        region (str): Language-region code (default: 'en-US').

    Returns:
        str: A summary of the top 3 news articles with title, summary, and link.
    """
    rapidapi_key = RAPIDAPI_KEY
    url = "https://google-news13.p.rapidapi.com/search"
    querystring = {"keyword": keyword, "lr": region}
    headers = {
        "x-rapidapi-key": rapidapi_key,
        "x-rapidapi-host": "google-news13.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        items = data.get("items", [])

        if not items:
            return f"No news found for keyword: {keyword}"

        result = f"📰 Top News for '{keyword}':\n"
        for idx, item in enumerate(items[:3], start=1):  # Top 3 items
            title = item.get("title", "No title")
            snippet = item.get("snippet", "No summary")
            link = item.get("newsUrl", "No link")
            result += f"\n{idx}. {title}\n📄 {snippet}\n🔗 {link}\n"
        return result.strip()

    except requests.exceptions.RequestException as e:
        return f"❌ Error fetching news: {str(e)}"
    except (KeyError, TypeError) as e:
        return f"❌ Unexpected response format: {str(e)}"


# Run google news agent.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agents.google_news_agent import GoogleNewsAgent
def main():
    agent = GoogleNewsAgent()
    query = "mental health"
    result = agent.run(query)
    print(result)

if __name__ == "__main__":
    main()

# Google News Agent py

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.spotify_tools import search_spotify_playlists
from config.setting import OPENAI_API_KEY

class GoogleNewsAgent:
    def __init__(self):
        self.openai_api_key = OPENAI_API_KEY
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,
            tools=[search_news_by_keyword],
            prompt = (
              "You are a helpful Google search assistant. Your job is to help users find Google articles "
              "based on their search queries. Use the tool to perform a search and return the top result. "
              "Once the top result is found, return it as 'Final Answer: <result>' and stop."
            ),
            name="google_news_assistant"
        )

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config={"recursion_limit": 50})
        return response["messages"][-1].content